## Applied Data Science Capstone Assignment 2 :Segmenting and Clustering Neighborhoods in Toronto 

### 1. Table of neighborhoods in Toronto

In [ ]:
# Import necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library




Solving environment: \ 

Link for scrapping a table from a website: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [3]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [5]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


#### Creating Pandas DataFrame

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[288, 288, 288]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [9]:
# Access the top 5 rows of the data frame 
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#### Dropping all cells with a borough that is Not assigned

In [11]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


#### Combining Neighbourhoods based on similar Postcode and Borough

In [23]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood\n'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge\n,Malvern"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union"
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [27]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [28]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [29]:
# Check the shape of the data frame
df.shape

(103, 3)

In [30]:
# save to dataframe
df.to_csv(r'df_canada.csv')

## 2. Longitude and latitude of the neighborhoods

In [ ]:
# Initialize varialbes
lat = []
lng = []
lat_lng_coords = None

# Get postcodes from neighborhoods table
postal_code = neighborhoods['Postcode']

# Store latitude and longitude values in lat and lng
for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])

In [ ]:
The latitude and longitude information, each stored in its own list, were added to the neighborhood dataframe.

In [ ]:
# Add the lists containing latitude and longitude values to the neighborhood dataframe
nh_complete = neighborhoods
nh_complete['Latitude'] = lat
nh_complete['Longitude'] = lng

In [ ]:
nh_complete.head()


## 3. Explore and cluster the neighborhoods

In [ ]:
# New dataframe with only the original city included
toronto = nh_complete[nh_complete['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
toronto.shape

A map of Toronto was created to visualize the city and all the neighborhoods

In [ ]:
# Get the latitude and longitude of Toronto
g = geocoder.arcgis('Toronto, Ontario')
lat_tor = g.latlng[0]
lng_tor = g.latlng[1]

# Create a map of Toronto
map_toronto = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Add markers to map
for lat, lng, bor, postcode in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Postcode']):
    label = '{}, {}'.format(postcode, bor)        # popup labels with postcode and borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
# Define Foursquare Credentials and Version
LIMIT = 100

CLIENT_ID = 'R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K' # your Foursquare ID
CLIENT_SECRET = '4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
 Limit the number of venues to 100
LIMIT = 100

# Get the venues for each postcode
toronto_venues = getNearbyVenues(names=toronto['Postcode'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude'],
                                 radius=700) # limit radius to 700 meters

In [ ]:
# Check the venues dataframe
print(toronto_venues.shape)
toronto_venues.head(10)

In [ ]:
# Check the number of venues for each postcode
num_venues = pd.DataFrame()
num_venues['Postcode'] = toronto_venues.groupby('Postcode').count().reset_index()['Postcode']
num_venues['Venue Count'] = toronto_venues.groupby('Postcode').count().reset_index()['Venue']
num_venues

To analyze each neighborhood and cluster them based on Foursquare venue categories, mean frequency of occurrences for each category was calculated for each neighborhood.

In [ ]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add Postcode column
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# Move postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(10)

Before clustering, a table was constructed with top venues for each neighborhood to show the characteristics of each neighborhood.

In [ ]:
# Define a function that returns the sorted categories
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]                                       # row of the table of interest
    row_categories_sorted = row_categories.sort_values(ascending=False) # sort the mean frequency of occurrences
    
    return row_categories_sorted.index.values[0:num_top_venues]         # return the index (category) of the mean frequency values

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postcode'] = toronto_grouped['Postcode']

# Each row of the new dataframe is filled with the sorted categories of each row of toronto_goruped dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

Finally, using k-means, the neighborhoods were clustered into 3 clusters. A map visualizing the clusters was also created.

In [ ]:
# Set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
 #Add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# Merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype('int')

toronto_merged.head()

In [ ]:
# Create map
map_clusters = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ', ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters